# Spacing Statistics

## 1. Importing / Installing Packages

In [2]:
import os # Importing os module for operating system dependent functionality

import pandas as pd # Importing pandas package

# Set the maximum number of columns to display to None
pd.set_option('display.max_columns', None)

import numpy as np # Importing numpy package

from typing import Dict, Tuple, List, Union, Optional # Importing specific types from typing module

from src.utils import DatabricksOdbcConnector # Importing DatabricksOdbcConnector class from database_manager module

from pyproj import Geod # Importing Geod from pyproj package

from tqdm import tqdm # Importing tqdm for progress bar functionality

from joblib import Parallel, delayed # Importing Parallel and delayed for parallel processing

from matplotlib import pyplot as plt # Importing pyplot from matplotlib for plotting

# Setting matplotlib to inline mode for Jupyter notebooks
%matplotlib inline

%config InlineBackend.figure_format = 'svg' # Configuring inline backend to use SVG format for figures

from src.well_data import WellDataLoader, GeoSurveyProcessor # Importing custom classes for well data management

# from src.utils import reorder_columns # Importing utility function to reorder DataFrame columns

## 2. Defining Functions

### 2.1. Defining Functions that is used in calculation for i-k pair dataframe and Spacing Stats

In [3]:
class WellSpacingCalculator:
    """
    Class for calculating well spacing metrics and directional relationships using
    3D lateral midpoint alignment and curvature-aware distances.
    Midpoints are projected in 2D space to remove lateral-length bias when calculating spacing.
    """

    def __init__(self, trajectories: Union[Dict[str, pd.DataFrame], pd.DataFrame]):
        if isinstance(trajectories, pd.DataFrame):
            if "uwi" not in trajectories.columns:
                raise ValueError("Trajectory DataFrame must contain 'uwi' column.")
            self._trajectory_df = trajectories.reset_index(drop=True)
            self.trajectories = {
                cid: group for cid, group in self._trajectory_df.groupby("uwi")
            }
        elif isinstance(trajectories, dict):
            self.trajectories = trajectories
            self._trajectory_df = pd.concat(
                trajectories.values(), keys=trajectories.keys()
            ).reset_index(drop=True)
        else:
            raise ValueError("Invalid type for trajectories. Must be DataFrame or Dict.")

    def _apply_vectorized_geod(
        self,
        lat1: np.ndarray,
        lon1: np.ndarray,
        lat2: np.ndarray,
        lon2: np.ndarray
    ) -> np.ndarray:
        """
        Vectorized geodetic distance calculation between two arrays of (lat1, lon1) and (lat2, lon2).
        Returns distance in feet.
        """
        geod = Geod(ellps="WGS84")
        _, _, dist_m = geod.inv(lon1, lat1, lon2, lat2)
        dist_ft = dist_m * 3.28084  # Convert meters to feet
        return dist_ft

    def _compute_normalized_midpoints(self, frac: float = 0.5, use_interpolation: bool = True) -> pd.DataFrame:
        """
        Computes midpoints for each well either by interpolating along the well trajectory
        using MD-based fractional position or by averaging heel and toe coordinates.

        Parameters:
        -----------
        frac : float
            Fractional location along the lateral to compute the midpoint (0.0 to 1.0).
        use_interpolation : bool
            If True, uses curvature-aware interpolation along MD.
            If False, uses geometric midpoint between heel and toe.

        Returns:
        --------
        pd.DataFrame indexed by 'uwi', containing:
            ['x', 'y', 'tvd', 'latitude', 'longitude']
        """
        df = self._trajectory_df.copy()
        df = df.sort_values(["uwi", "md"]).reset_index(drop=True)

        if not use_interpolation:
            # Simple geometric midpoint (fast)
            heel_toe_df = (
                df.groupby("uwi")
                .agg(
                    heel_x=("x", "first"),
                    heel_y=("y", "first"),
                    heel_tvd=("tvd", "first"),
                    heel_lat=("latitude", "first"),
                    heel_lon=("longitude", "first"),
                    toe_x=("x", "last"),
                    toe_y=("y", "last"),
                    toe_tvd=("tvd", "last"),
                    toe_lat=("latitude", "last"),
                    toe_lon=("longitude", "last"),
                )
            )

            midpoint_df = pd.DataFrame({
                "x": (heel_toe_df["heel_x"] + heel_toe_df["toe_x"]) / 2,
                "y": (heel_toe_df["heel_y"] + heel_toe_df["toe_y"]) / 2,
                "tvd": (heel_toe_df["heel_tvd"] + heel_toe_df["toe_tvd"]) / 2,
                "latitude": (heel_toe_df["heel_lat"] + heel_toe_df["toe_lat"]) / 2,
                "longitude": (heel_toe_df["heel_lon"] + heel_toe_df["toe_lon"]) / 2,
            })
            midpoint_df.index.name = "uwi"
            return midpoint_df

        # Interpolated midpoint (MD-based)
        min_md = df.groupby("uwi")["md"].transform("min")
        max_md = df.groupby("uwi")["md"].transform("max")
        df["normalized_md"] = (df["md"] - min_md) / (max_md - min_md)

        df["row_index"] = df.groupby("uwi").cumcount()
        df["prev_idx"] = df.groupby("uwi")["normalized_md"].transform(lambda x: x.searchsorted(frac, side="right") - 1)
        df["next_idx"] = df["prev_idx"] + 1
        df["next_idx"] = np.minimum(df["next_idx"], df["row_index"].groupby(df["uwi"]).transform("max"))

        df_prev = df.groupby("uwi").apply(lambda g: g.loc[g["row_index"] == g["prev_idx"].iloc[0]]).reset_index(drop=True)
        df_next = df.groupby("uwi").apply(lambda g: g.loc[g["row_index"] == g["next_idx"].iloc[0]]).reset_index(drop=True)

        merged = pd.merge(df_prev, df_next, on="uwi", suffixes=("_prev", "_next"))

        delta = merged["normalized_md_next"] - merged["normalized_md_prev"]
        delta = delta.replace(0, np.nan)
        ratio = (frac - merged["normalized_md_prev"]) / delta

        midpoint_df = pd.DataFrame({
            "x": merged["x_prev"] + ratio * (merged["x_next"] - merged["x_prev"]),
            "y": merged["y_prev"] + ratio * (merged["y_next"] - merged["y_prev"]),
            "tvd": merged["tvd_prev"] + ratio * (merged["tvd_next"] - merged["tvd_prev"]),
            "latitude": merged["latitude_prev"] + ratio * (merged["latitude_next"] - merged["latitude_prev"]),
            "longitude": merged["longitude_prev"] + ratio * (merged["longitude_next"] - merged["longitude_prev"]),
        })
        midpoint_df["uwi"] = merged["uwi"]
        return midpoint_df.set_index("uwi")

    def _compute_drill_directions(self) -> pd.Series:
        median_azimuth = self._trajectory_df.groupby("uwi")["azimuth"].median()
        is_ew = ((median_azimuth >= 45) & (median_azimuth <= 135)) | ((median_azimuth >= 225) & (median_azimuth <= 315))
        return pd.Series(np.where(is_ew, "EW", "NS"), index=median_azimuth.index, name="drill_direction")

    def _filter_close_pairs(self, lat: np.ndarray, lon: np.ndarray, max_distance_miles: float = 20.0) -> Tuple[np.ndarray, np.ndarray]:

        lat1, lat2 = np.meshgrid(lat, lat, indexing="ij")
        lon1, lon2 = np.meshgrid(lon, lon, indexing="ij")

        delta_lat = np.abs(lat1 - lat2)
        delta_lon = np.abs(lon1 - lon2)

        miles_per_lat_degree = 69.0
        miles_per_lon_degree = 69.0 * np.cos(np.radians(lat))
        miles_per_lon_degree_matrix = np.add.outer(miles_per_lon_degree, miles_per_lon_degree) / 2.0

        rough_dist_miles = np.sqrt(
            (delta_lat * miles_per_lat_degree)**2 + (delta_lon * miles_per_lon_degree_matrix)**2
        )

        mask = (rough_dist_miles <= max_distance_miles) & (delta_lat + delta_lon > 0)
        i_idx, k_idx = np.where(mask)

        return i_idx, k_idx

    def _get_pairwise_indices(self, uwis: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Generate all valid pairwise (i, k) UWI combinations from an array of well IDs,
        excluding self-comparisons (i != k).

        Parameters
        ----------
        uwis : np.ndarray
            Array of unique well identifiers.

        Returns
        -------
        Tuple[np.ndarray, np.ndarray]
            Two 1D arrays of i and k UWIs representing all valid (i, k) pairs.
        """
        # Generate meshgrid of al possible UWI pairs
        n = len(uwis)
        i_idx, k_idx = np.meshgrid(np.arange(n), np.arange(n), indexing="ij")
            
        # Exclude self-comparisons (where i_uwi == k_uwi)
        valid_mask = i_idx != k_idx

        return i_idx[valid_mask], k_idx[valid_mask]
    
    def _get_relative_cardinal_direction(
        self,
        lat: np.ndarray,
        lon: np.ndarray,
        i_idx: np.ndarray,
        k_idx: np.ndarray
    ) -> np.ndarray:
        lat1, lat2 = lat[i_idx], lat[k_idx]
        lon1, lon2 = lon[i_idx], lon[k_idx]

        lat_diff = lat1 - lat2
        lon_diff = lon1 - lon2

        vertical = np.abs(lat_diff) > np.abs(lon_diff)
        is_south = lat_diff > 0
        is_west = lon_diff > 0

        return np.select(
            [vertical & is_south, vertical & ~is_south, ~vertical & is_west, ~vertical & ~is_west],
            ["S", "N", "W", "E"]
        )
    
    def _process_batch(self, i_idx: np.ndarray, k_idx: np.ndarray, ids: np.ndarray, coords: np.ndarray, lat_lon: np.ndarray, directions: np.ndarray, curvature_threshold_ft: float, use_geod: bool) -> pd.DataFrame:
        i_uwi = ids[i_idx]
        k_uwi = ids[k_idx]

        horizontal, vertical, dist3d = self._calculate_distances(coords, lat_lon, i_idx, k_idx, curvature_threshold_ft, use_geod)
        direction_to_k_from_i  = self._get_relative_cardinal_direction(lat_lon[:, 0], lat_lon[:, 1], i_idx, k_idx)

        return pd.DataFrame({
            "well_i": i_uwi,
            "well_k": k_uwi,
            "horizontal_dist": horizontal,
            "vertical_dist": vertical,
            "3D_dist": dist3d,
            "drill_direction_i": directions[i_idx],
            "drill_direction_k": directions[k_idx],
            "direction_to_k_from_i": direction_to_k_from_i
        })
    
    def _batch_filtered_indices(self, pairs: List[Tuple[int, int]], batch_size: int = 1_000_000):
        """
        Vectorized batching of prefiltered well pairs.

        Parameters
        ----------
        pairs : List[Tuple[int, int]]
            List of (i_idx, k_idx) pairs.
        batch_size : int
            Number of pairs per batch.

        Yields
        ------
        Tuple[np.ndarray, np.ndarray]
            i_idx and k_idx arrays for each batch.
        """
        pairs_array = np.array(pairs)  # Convert list of tuples directly to 2D array (N, 2)
        n_pairs = pairs_array.shape[0]

        # Vectorized slicing
        split_indices = np.arange(0, n_pairs, batch_size)

        for start_idx in split_indices:
            end_idx = min(start_idx + batch_size, n_pairs)
            batch = pairs_array[start_idx:end_idx]
            yield batch[:, 0], batch[:, 1]

    def _calculate_distances(
    self,
    coords: np.ndarray,
    lat_lon: np.ndarray,
    i_idx: np.ndarray,
    k_idx: np.ndarray,
    curvature_threshold_ft: float,
    use_geod: bool
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        Calculates horizontal, vertical, and 3D distances between well pairs
        based on normalized midpoint coordinates.
        """
        A = coords[i_idx]  # shape (N, 3): x, y, tvd
        B = coords[k_idx]

        B_aligned = B.copy()

        B_aligned[:, 1] = A[:, 1]  # Force y of B to match y of A

        dx = B_aligned[:, 0] - A[:, 0]
        dy = B_aligned[:, 1] - A[:, 1]
        dz = B_aligned[:, 2] - A[:, 2]

        horizontal = np.sqrt(dx**2 + dy**2)
        vertical = np.abs(dz)
        dist3d = np.sqrt(horizontal**2 + vertical**2)

        if use_geod:
            long_mask = horizontal > curvature_threshold_ft
            if np.any(long_mask):
                lat = lat_lon[:, 0]
                lon = lat_lon[:, 1]
                lat1 = lat[i_idx[long_mask]]
                lon1 = lon[i_idx[long_mask]]
                lat2 = lat[k_idx[long_mask]]
                lon2 = lon[k_idx[long_mask]]

                # 🔥 Now call the optimized function!
                horizontal[long_mask] = self._apply_vectorized_geod(lat1, lon1, lat2, lon2)

                # Update 3D distance with corrected horizontal
                dist3d[long_mask] = np.sqrt(horizontal[long_mask]**2 + vertical[long_mask]**2)

        return horizontal, vertical, dist3d

    def _calculate_spacing_statistics(
    self,
    curvature_threshold_ft: float = 26400.0,
    use_geod: bool = True,
    frac: float = 0.5,
    batch_size: int = 1_000_000,
    max_distance_miles: Optional[float] = 20.0,
    save_batches_dir: Optional[str] = None,
    use_interpolation: bool = True
) -> Optional[pd.DataFrame]:
        """
        Compute spacing distances between well pairs using lateral midpoints.

        Parameters:
        -----------
        curvature_threshold_ft : float
            Use geodetic correction above this horizontal threshold (default 5 miles).
        use_geod : bool
            Whether to apply geodetic distance correction.
        frac : float
            Fractional position along the lateral for midpoint interpolation (default 0.5).
        batch_size : int
            Number of pairwise calculations to process per batch.
        max_distance_miles : float or None
            Max rough distance filter to exclude far-away well pairs.
        save_batches_dir : str or None
            If provided, saves each batch to a separate Parquet file.
        use_interpolation : bool
            Whether to interpolate midpoints based on MD (curvature-aware).
            If False, computes geometric midpoints between heel and toe.

        Returns:
        --------
        pd.DataFrame or None
            If saving to disk, returns None. Otherwise, returns spacing results DataFrame.
        """
        # 1. Get normalized lateral midpoints and drill directions
        midpoint_df = self._compute_normalized_midpoints(frac=frac, use_interpolation=use_interpolation)
        drill_dirs = self._compute_drill_directions()
        midpoint_df["drill_direction"] = drill_dirs

        # 2. Prepare arrays
        ids = midpoint_df.index.to_numpy()
        coords = midpoint_df[["x", "y", "tvd"]].to_numpy()
        lat_lon = midpoint_df[["latitude", "longitude"]].to_numpy()
        directions = midpoint_df["drill_direction"].to_numpy()

        if max_distance_miles is not None:
            lat = lat_lon[:, 0]
            lon = lat_lon[:, 1]
            i_idx, k_idx = self._filter_close_pairs(lat, lon, max_distance_miles)
        else:
            i_idx, k_idx = self._get_pairwise_indices(ids)

        pairs = list(zip(i_idx, k_idx))
        batch_generator = list(self._batch_filtered_indices(pairs, batch_size=batch_size))
        n_batches = len(batch_generator)

        if save_batches_dir:
            os.makedirs(save_batches_dir, exist_ok=True)

        def process_and_save(batch_number: int, i_idx: np.ndarray, k_idx: np.ndarray):
            
            batch_df = self._process_batch(i_idx, k_idx, ids, coords, lat_lon, directions, curvature_threshold_ft, use_geod)
            
            if save_batches_dir:
                filepath = os.path.join(save_batches_dir, f"spacing_batch_{batch_number:04d}.parquet")
                batch_df.to_parquet(filepath, index=False)
            return batch_df
        
        tqdm_kwargs = {
            "desc": "🚀 Calculating Spacing (Parallel)",
            "dynamic_ncols": True, # Auto-adjust width to terminal
            "smoothing": 0.3, # Smoothing factor for progress bar
            "bar_format": "{desc}: |{bar:40}| {percentage:3.0f}% {n_fmt}/{total_fmt} [{elapsed}<{remaining}]", # Custom bar format
            "ascii": "░▒█", # Use custom ASCII characters for the bar
            "leave": True, # Leave the progress bar on completion
        }
        
        results = Parallel(n_jobs=-1)(
            delayed(process_and_save)(batch_num, i_idx, k_idx)
            for batch_num, (i_idx, k_idx) in tqdm(enumerate(batch_generator), total=n_batches, **tqdm_kwargs)
        )

        if save_batches_dir:
            print(f"✅ All batches saved to {save_batches_dir}")
            return None
        else:
            return pd.concat(results, ignore_index=True)

    def _load_saved_batches(self, batch_folder: str) -> pd.DataFrame:
        """
        Load all saved spacing batch Parquet files from a folder and combine into a single DataFrame.

        Parameters
        ----------
        batch_folder : str
            Path to the folder where batch Parquet files are stored.

        Returns
        -------
        pd.DataFrame
            Combined spacing DataFrame.
        """
        if not os.path.isdir(batch_folder):
            raise FileNotFoundError(f"Batch folder '{batch_folder}' not found.")

        batch_files = sorted([
            os.path.join(batch_folder, f)
            for f in os.listdir(batch_folder)
            if f.endswith(".parquet")
        ])

        if not batch_files:
            raise ValueError(f"No Parquet files found in folder '{batch_folder}'.")

        print(f"🔍 Found {len(batch_files)} batch files. Loading and combining...")

        dfs = []
        for file in batch_files:
            dfs.append(pd.read_parquet(file))

        combined_df = pd.concat(dfs, ignore_index=True)
        print(f"✅ Loaded {len(combined_df):,} rows from all batches.")
        return combined_df

### 2.1. Defining Functions that is used in unified purpose of computing well bounding relationships from both parent-child logic and physical overlap.

In [4]:
class WellBoundingAnalyzer:
    def __init__(
        self,
        spacing_df: pd.DataFrame,
        trajectory_data: Union[pd.DataFrame, Dict[str, pd.DataFrame]],
        header_df: pd.DataFrame,
        horiz_cutoff_bounds: float = 1800,
        vert_cutoff_bounds: float = 125,
        horiz_cutoff_parent: float = 1320.0,
        overlap_threshold: float = 30.0
    ):
        # Filter only E/W direction to begin with
        spacing_df = spacing_df[spacing_df["direction_to_k_from_i"].isin(["E", "W"])].copy()

        self.spacing_df = spacing_df.copy()
        self.trajectory_data = self._prepare_trajectory_dict(trajectory_data)
        self.horiz_cutoff_bounds = horiz_cutoff_bounds
        self.vert_cutoff_bounds = vert_cutoff_bounds
        self.horiz_cutoff_parent = horiz_cutoff_parent
        self.overlap_threshold = overlap_threshold

        headers = header_df.set_index("uwi")
        for suffix in ["i", "k"]:
            self.spacing_df[f"completion_date_{suffix}"] = self.spacing_df[f"well_{suffix}"].map(headers["comp_date"])

        self._validate_inputs()

    def _validate_inputs(self) -> None:
        required_columns = [
            "well_i", "well_k", "horizontal_dist", "vertical_dist",
            "completion_date_i", "completion_date_k",
            "drill_direction_i", "drill_direction_k",
            "direction_to_k_from_i"
        ]
        missing = [col for col in required_columns if col not in self.spacing_df.columns]
        if missing:
            raise ValueError(f"Missing columns in spacing_df: {missing}")
        self.spacing_df["completion_date_i"] = pd.to_datetime(self.spacing_df["completion_date_i"])
        self.spacing_df["completion_date_k"] = pd.to_datetime(self.spacing_df["completion_date_k"])

    def _prepare_trajectory_dict(self, data: Union[pd.DataFrame, Dict[str, pd.DataFrame]]) -> Dict[str, pd.DataFrame]:
        if isinstance(data, dict):
            return data
        elif isinstance(data, pd.DataFrame):
            if "uwi" not in data.columns or "md" not in data.columns:
                raise ValueError("Trajectory DataFrame must contain 'uwi' and 'md' columns.")
            return {
                uwi: df.drop(columns="uwi").reset_index(drop=True)
                for uwi, df in data.groupby("uwi")
            }
        else:
            raise ValueError("Invalid trajectory data input type.")

    def get_valid_parent_child_pairs(self) -> pd.DataFrame:
        df = self.spacing_df
        horiz_ok = df["horizontal_dist"] <= self.horiz_cutoff_parent
        vert_ok = True if self.vert_cutoff_bounds is None else df["vertical_dist"] <= self.vert_cutoff_bounds
        time_ok = df["completion_date_i"] < df["completion_date_k"]
        dir_ok = df["drill_direction_i"] == df["drill_direction_k"]

        cond = horiz_ok & time_ok & dir_ok
        if self.vert_cutoff_bounds is not None:
            cond = cond & vert_ok

        return df[cond].copy()

    def get_closest_parent_per_child(self) -> pd.DataFrame:
        valid_pairs = self.get_valid_parent_child_pairs()
        idx_closest = valid_pairs.groupby("well_k")["horizontal_dist"].idxmin()
        closest_df = valid_pairs.loc[idx_closest].copy()
        return closest_df.rename(columns={
            "well_i": "parent_uwi",
            "well_k": "child_uwi"
        })[[
            "parent_uwi", "child_uwi", "horizontal_dist", "vertical_dist",
            "completion_date_i", "completion_date_k", "drill_direction_i"
        ]]

    def get_bound_category_by_count(self) -> pd.DataFrame:
        valid_pairs = self.get_valid_parent_child_pairs()
        parent_counts = valid_pairs.groupby("well_k").size().reset_index(name="parent_count")
        parent_counts["bound_category_by_count"] = parent_counts["parent_count"].map({
            0: "Unbounded",
            1: "Single-Side Bounded"
        }).fillna("Multi-Side Bounded")
        return parent_counts.rename(columns={"well_k": "child_uwi"})

    def _calculate_overlap(self, well_A: pd.DataFrame, well_B: pd.DataFrame) -> float:
        if well_A.empty or well_B.empty:
            return 0.0
        start_A, end_A = well_A["md"].min(), well_A["md"].max()
        start_B, end_B = well_B["md"].min(), well_B["md"].max()
        overlap_start = max(start_A, start_B)
        overlap_end = min(end_A, end_B)
        if overlap_start >= overlap_end:
            return 0.0
        overlap_length = overlap_end - overlap_start
        shorter_length = min(end_A - start_A, end_B - start_B)
        return (overlap_length / shorter_length) * 100 if shorter_length > 0 else 0.0

    def get_overlap_bounds(self) -> pd.DataFrame:
        
        df = self.spacing_df.copy()
        df = df.sort_values(["well_i", "direction_to_k_from_i", "horizontal_dist"])
        closest_df = df.groupby(["well_i", "direction_to_k_from_i"]).first()
        all_well_ids = self.spacing_df["well_i"].unique()
        output = pd.DataFrame(index=pd.Index(all_well_ids, name="well_i"))

        results = []
        for direction, suffix in zip(["E", "W"], ["1", "2"]):
            try:
                dir_df = closest_df.xs(direction, level="direction_to_k_from_i", drop_level=False)
            except KeyError:
                continue

            overlaps, uwis, dirs, idxs = [], [], [], []

            for well_key, row in dir_df.iterrows():
                well_i = well_key[0] if isinstance(well_key, tuple) else well_key
                well_k = row["well_k"]
                h_dist = row["horizontal_dist"]
                v_dist = row["vertical_dist"]

                h_ok = h_dist <= self.horiz_cutoff_bounds
                v_ok = True if self.vert_cutoff_bounds is None else v_dist <= self.vert_cutoff_bounds

                if h_ok and v_ok:
                    traj_i = self.trajectory_data.get(str(well_i), pd.DataFrame())
                    traj_k = self.trajectory_data.get(str(well_k), pd.DataFrame())
                    overlap = self._calculate_overlap(traj_i, traj_k)
                    overlaps.append(overlap)
                    uwis.append(str(well_k))
                    dirs.append(direction)
                else:
                    overlaps.append(None)
                    uwis.append(None)
                    dirs.append(None)

                idxs.append(well_i)

            partial_df = pd.DataFrame({
                f"bound_{suffix}": overlaps,
                f"bound_{suffix}_uwi": uwis,
                f"bound_{suffix}_direction": dirs
            }, index=idxs)

            output = output.join(partial_df, how="left")

        swap = output["bound_2"].fillna(0) > output["bound_1"].fillna(0)
        for col in ["bound_1", "bound_1_uwi", "bound_1_direction"]:
            alt = col.replace("1", "2")
            output[col], output[alt] = (
                output[alt].where(swap, output[col]),
                output[col].where(swap, output[alt])
            )

        output.index.name = "well_i"
        return output.reset_index()

    def _label_overlap_category(self, df: pd.DataFrame) -> pd.DataFrame:
        def label(row):
            if pd.isna(row["bound_1"]):
                return "Unbounded"
            elif row["bound_1"] >= self.overlap_threshold:
                return "Strongly Bounded"
            elif row["bound_1"] > 0:
                return "Weakly Bounded"
            return "Unbounded"

        df["bound_category_by_overlap"] = df.apply(label, axis=1)
        return df

    def get_full_bound_summary(self) -> pd.DataFrame:
        overlap_bounds = self.get_overlap_bounds()
        overlap_bounds = self._label_overlap_category(overlap_bounds)

        closest_parents = self.get_closest_parent_per_child()
        parent_categories = self.get_bound_category_by_count()

        summary = overlap_bounds.merge(closest_parents, left_on="well_i", right_on="child_uwi", how="left")
        summary = summary.merge(parent_categories, on="child_uwi", how="left")

        return summary

## 3. Loading Header and GeoSurvey either from Excel/csv/SQL into Pandas DataFrame

In [5]:
loader = WellDataLoader(db = DatabricksOdbcConnector(), 
                        log_dir=r"C:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\Python\Parent_Child_Spacing\logs")

In [6]:
df_MB_header = loader.get_header_data(basin="MB", start_year=2014)

[WellDataLoaderLogger] INFO (05-05 09:06 PM): Loading header data from SQL. (Line: 84) [well_data_manager.py]

c:\users\apoorva.saxena\onedrive - sitio royalties\desktop\project - apoorva\python\parent_child_spacing\src\utils\database_manager.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(sql_query, self.connection)


In [7]:
df_MB_directional = loader.get_directional_data()

[WellDataLoaderLogger] INFO (05-05 09:06 PM): Loading directional data from SQL. (Line: 104) [well_data_manager.py]



In [8]:
processor = GeoSurveyProcessor(log_dir=r"C:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\Python\Parent_Child_Spacing\logs")

[GeoLogger] INFO (05-05 09:13 PM): GeoSurveyProcessor initialized. (Line: 186) [well_data_manager.py]



In [9]:
df_utm = processor.compute_utm_coordinates(df=df_MB_directional)

[GeoLogger] INFO (05-05 09:13 PM): ✅ Using lat/lon from input DataFrame. (Line: 262) [well_data_manager.py]

[GeoLogger] INFO (05-05 09:13 PM): ✅ UTM coordinate computation complete in 26.10 sec. (Line: 317) [well_data_manager.py]



In [10]:
df_utm_lateral = processor.filter_after_heel_point(df=df_utm)

In [11]:
spacing_calculator = WellSpacingCalculator(trajectories=df_utm_lateral)

In [12]:
# spacing_calculator._calculate_spacing_statistics(
#     batch_size=500_000,
#     max_distance_miles=10,  # Adjust as needed for your dataset
#     save_batches_dir = "spacing_batches_MB_2016",  # Specify directory to save batches,
#     use_interpolation=False  # Set to True if you want to use MD-based interpolation for midpoints
# )

In [13]:
# df_spacing_new['well_i_10'] = df_spacing_new['well_i'].str[:10] # Extracting first 10 characters of 'well_i' column
# df_spacing_new['well_k_10'] = df_spacing_new['well_k'].str[:10] # Extracting first 10 characters of 'well_k' column
# df_spacing_new = reorder_columns(df_spacing_new, ['well_i_10','well_k_10'], 'well_k') # Reordering columns to have 'well_i_10' and 'well_k_10' at the beginning

# df_spacing_old['well_i_10'] = df_spacing_old['well_i'].str[:10] # Extracting first 10 characters of 'well_i' column
# df_spacing_old['well_k_10'] = df_spacing_old['well_k'].str[:10] # Extracting first 10 characters of 'well_k' column
# df_spacing_old = reorder_columns(df_spacing_old, ['well_i_10','well_k_10'], 'well_k') # Reordering columns to have 'well_i_10' and 'well_k_10' at the beginning

## 4. Testing

In [14]:
df_spacing_new = spacing_calculator._load_saved_batches(batch_folder="spacing_batches_MB_2016") # Loading the saved batches into a DataFrame

🔍 Found 65 batch files. Loading and combining...
✅ Loaded 32,127,528 rows from all batches.


In [20]:
analyzer = WellBoundingAnalyzer(
    spacing_df = df_spacing_new,
    trajectory_data = df_utm_lateral,
    header_df = df_MB_header,
    horiz_cutoff_bounds = 1800,
    vert_cutoff_bounds = 125,       # <- can be None now
    horiz_cutoff_parent = 1320,
    overlap_threshold = 30.0
)

In [21]:
bounds_df = analyzer.get_overlap_bounds()

In [32]:
bounds_df

,well_i,bound_1,bound_1_uwi,bound_1_direction,bound_2,bound_2_uwi,bound_2_direction
0,42003073100100,NaN,None,None,NaN,None,None
1,42003442780000,NaN,None,None,NaN,None,None
2,42003452020000,NaN,None,None,NaN,None,None
3,42003455230000,NaN,None,None,NaN,None,None
4,42003455330000,NaN,None,None,NaN,None,None
...,...,...,...,...,...,...,...
23491,42501375310000,100.000000,42501375300000,W,99.305556,42501375320000,E
23492,42501375320000,99.305556,42501375310000,W,NaN,None,None
23493,42501375330000,NaN,None,None,NaN,None,None
23494,42501375540000,97.444946,42501368370000,E,96.812526,42501366520000,W


In [15]:
def calculate_overlap(well_A: pd.DataFrame, well_B: pd.DataFrame) -> float:
    """
    Calculate the percentage of measured depth (MD) overlap between two horizontal wellbores.

    This function computes the length of the overlapping interval between the two wells'
    measured depth ranges and expresses it as a percentage of the shorter lateral length.
    It is typically used to assess lateral adjacency between horizontal wells.

    Parameters
    ----------
    well_A : pd.DataFrame
        Trajectory data for the first well. Must contain a column named 'md' representing measured depth (in feet).
        The DataFrame should represent the lateral portion of the well only.
    
    well_B : pd.DataFrame
        Trajectory data for the second well. Must contain a column named 'md' representing measured depth (in feet).
        The DataFrame should represent the lateral portion of the well only.

    Returns
    -------
    float
        The percentage of overlap between the two MD intervals, relative to the shorter well's lateral length.
        Returns 0.0 if either well is empty, lacks 'MD', or has no overlapping interval.

    Examples
    --------
    >>> calculate_overlap(well_A_df, well_B_df)
    67.5
    """

    start_A, end_A = well_A["md"].min(), well_A["md"].max()
    start_B, end_B = well_B["md"].min(), well_B["md"].max()

    overlap_start = max(start_A, start_B)
    overlap_end = min(end_A, end_B)

    if overlap_start >= overlap_end:
        return 0.0

    overlap_length = overlap_end - overlap_start
    shorter_length = min(end_A - start_A, end_B - start_B)

    return (overlap_length / shorter_length) * 100 if shorter_length > 0 else 0.0

def calculate_lateral_adjacency_bounds(
    spacing_df: pd.DataFrame,
    trajectory_data: Union[pd.DataFrame, Dict[str, pd.DataFrame]],
    vertical_cutoff: float = 125.0,
    horizontal_cutoff: float = 1800.0
) -> pd.DataFrame:
    """
    For each well_i, find the closest E and W well_k neighbors by horizontal distance.
    Compute lateral overlap % only if vertical/horizontal cutoffs are satisfied.
    Returns the highest overlap as bound_1 and the second as bound_2.

    Parameters
    ----------
    spacing_df : pd.DataFrame
        Spacing output from WellSpacingCalculator, with at least:
        ['well_i', 'well_k', 'horizontal_dist', 'vertical_dist', 'direction_to_k_from_i']
    
    trajectory_data : Union[pd.DataFrame, Dict[str, pd.DataFrame]]
        Either:
            - A pre-filtered lateral-only DataFrame with a 'uwi' column and 'md'
            - OR a dictionary mapping UWI (str) to its corresponding trajectory DataFrame
    
    vertical_cutoff : float
        Maximum vertical distance (ft) allowed to compute lateral overlap
    
    horizontal_cutoff : float
        Maximum horizontal distance (ft) allowed to compute lateral overlap

    Returns
    -------
    pd.DataFrame
        Contains:
        ['well_i', 'bound_1', 'bound_1_uwi', 'bound_1_direction',
                     'bound_2', 'bound_2_uwi', 'bound_2_direction']
    """
    # Step 1: Convert lateral DataFrame to dict if needed
    if isinstance(trajectory_data, pd.DataFrame):
        if "uwi" not in trajectory_data.columns or "md" not in trajectory_data.columns:
            raise ValueError("Trajectory DataFrame must contain 'uwi' and 'md' columns.")
        
        trajectory_dict = {
            uwi: df.drop(columns=["uwi"], errors="ignore").reset_index(drop=True)
            for uwi, df in trajectory_data.groupby("uwi")
        }
    elif isinstance(trajectory_data, dict):
        trajectory_dict = trajectory_data
    else:
        raise ValueError("trajectory_data must be a DataFrame or a dictionary.")

    # Step 2: Filter for E/W directions only
    ew_df = spacing_df[spacing_df["direction_to_k_from_i"].isin(["E", "W"])].copy()

    # Step 3: Sort by well_i, direction, horizontal_dist to find nearest neighbor
    ew_df = ew_df.sort_values(["well_i", "direction_to_k_from_i", "horizontal_dist"])

    # Step 4: Get closest E and W well_k for each well_i
    closest_df = ew_df.groupby(["well_i", "direction_to_k_from_i"]).first()
    all_well_ids = spacing_df["well_i"].unique()
    output = pd.DataFrame(index=pd.Index(all_well_ids, name="well_i"))

    for direction, suffix in zip(["E", "W"], ["1", "2"]):
        try:
            dir_df = closest_df.xs(direction, level="direction_to_k_from_i", drop_level=False)
        except KeyError:
            # No wells in this direction
            continue

        overlaps, uwis, dirs, idxs = [], [], [], []

        for well_key, row in dir_df.iterrows():
            well_i = well_key[0] if isinstance(well_key, tuple) else well_key
            well_k = row["well_k"]
            h_dist, v_dist = row["horizontal_dist"], row["vertical_dist"]

            if h_dist <= horizontal_cutoff and v_dist <= vertical_cutoff:
                traj_i = trajectory_dict.get(str(well_i), pd.DataFrame())
                traj_k = trajectory_dict.get(str(well_k), pd.DataFrame())
                percent = calculate_overlap(traj_i, traj_k)
                overlaps.append(percent)
                uwis.append(str(well_k))
                dirs.append(direction)
            else:
                overlaps.append(None)
                uwis.append(None)
                dirs.append(None)

            idxs.append(well_i)

        # Construct and merge partial result
        partial_df = pd.DataFrame({
            f"bound_{suffix}": overlaps,
            f"bound_{suffix}_uwi": uwis,
            f"bound_{suffix}_direction": dirs
        }, index=idxs)

        output = output.join(partial_df, how="left")

    # Step 5: Reorder so that bound_1 is the higher of the two
    reordered = output.copy()
    swap_mask = reordered["bound_2"].fillna(0) > reordered["bound_1"].fillna(0)

    for col in ["bound_1", "bound_1_uwi", "bound_1_direction"]:
        alt_col = col.replace("1", "2")
        reordered[col], reordered[alt_col] = (
            reordered[alt_col].where(swap_mask, reordered[col]),
            reordered[col].where(swap_mask, reordered[alt_col])
        )

    return reordered.reset_index()

In [41]:
def calculate_lateral_overlap_metrics(
    well_A: pd.DataFrame,
    well_B: pd.DataFrame
) -> Tuple[float, float, float]:
    """
    Compute directional and union-based MD overlap metrics between two horizontal wells.

    Parameters
    ----------
    well_A : pd.DataFrame
        Directional survey data for Well A. Must contain 'md' column.
    well_B : pd.DataFrame
        Directional survey data for Well B. Must contain 'md' column.

    Returns
    -------
    Tuple[float, float, float]
        - % overlap relative to Well A's lateral length
        - % overlap relative to Well B's lateral length
        - % overlap relative to the union of both wells' MD ranges

        If there is no overlap, returns (0.0, 0.0, 0.0)
    """
    start_A, end_A = well_A['md'].min(), well_A['md'].max()
    start_B, end_B = well_B['md'].min(), well_B['md'].max()

    overlap_start = max(start_A, start_B)
    overlap_end = min(end_A, end_B)

    if overlap_start >= overlap_end:
        return 0.0, 0.0, 0.0

    overlap_length = overlap_end - overlap_start
    len_A = end_A - start_A
    len_B = end_B - start_B
    union_length = max(end_A, end_B) - min(start_A, start_B)

    overlap_pct_A = (overlap_length / len_A) * 100 if len_A > 0 else 0.0
    overlap_pct_B = (overlap_length / len_B) * 100 if len_B > 0 else 0.0
    overlap_pct_union = (overlap_length / union_length) * 100 if union_length > 0 else 0.0

    return overlap_pct_A, overlap_pct_B, overlap_pct_union

def calculate_lateral_adjacency_bounds(
    spacing_df: pd.DataFrame,
    trajectory_data: Union[pd.DataFrame, Dict[str, pd.DataFrame]],
    vertical_cutoff: float = 125.0,
    horizontal_cutoff: float = 1800.0,
    overlap_type: str = "directional"  # can be "directional" or "union"
) -> pd.DataFrame:
    """
    For each well_i, find the closest E and W well_k neighbors by horizontal distance.
    Compute lateral overlap % only if vertical/horizontal cutoffs are satisfied.
    Returns the highest overlap as bound_1 and the second as bound_2.

    Parameters
    ----------
    spacing_df : pd.DataFrame
        Output from WellSpacingCalculator with ['well_i', 'well_k', 'horizontal_dist', 'vertical_dist', 'direction_to_k_from_i']
    trajectory_data : Union[pd.DataFrame, Dict[str, pd.DataFrame]]
        Either a DataFrame with 'uwi' and 'md', or a dictionary mapping UWI to trajectory DataFrames.
    vertical_cutoff : float
        Max allowed vertical distance (ft) to consider for overlap.
    horizontal_cutoff : float
        Max allowed horizontal distance (ft) to consider for overlap.
    overlap_type : str
        Either "directional" (well_i normalized) or "union" (total MD span normalized)

    Returns
    -------
    pd.DataFrame
        Columns include:
        ['well_i', 'bound_1', 'bound_1_uwi', 'bound_1_direction',
                     'bound_2', 'bound_2_uwi', 'bound_2_direction']
    """
    # Normalize trajectory input
    if isinstance(trajectory_data, pd.DataFrame):
        if "uwi" not in trajectory_data.columns or "md" not in trajectory_data.columns:
            raise ValueError("Trajectory DataFrame must contain 'uwi' and 'md' columns.")
        trajectory_dict = {
            uwi: df.drop(columns=["uwi"], errors="ignore").reset_index(drop=True)
            for uwi, df in trajectory_data.groupby("uwi")
        }
    elif isinstance(trajectory_data, dict):
        trajectory_dict = trajectory_data
    else:
        raise ValueError("trajectory_data must be a DataFrame or a dictionary.")

    # Filter for E/W neighbors
    ew_df = spacing_df[spacing_df["direction_to_k_from_i"].isin(["E", "W"])].copy()
    ew_df = ew_df.sort_values(["well_i", "direction_to_k_from_i", "horizontal_dist"])
    closest_df = ew_df.groupby(["well_i", "direction_to_k_from_i"]).first()
    all_well_ids = spacing_df["well_i"].unique()
    output = pd.DataFrame(index=pd.Index(all_well_ids, name="well_i"))

    for direction, suffix in zip(["E", "W"], ["1", "2"]):
        try:
            dir_df = closest_df.xs(direction, level="direction_to_k_from_i", drop_level=False)
        except KeyError:
            continue

        overlaps, uwis, dirs, idxs = [], [], [], []

        for well_key, row in dir_df.iterrows():
            well_i = well_key[0] if isinstance(well_key, tuple) else well_key
            well_k = row["well_k"]
            h_dist, v_dist = row["horizontal_dist"], row["vertical_dist"]

            if h_dist <= horizontal_cutoff and v_dist <= vertical_cutoff:
                traj_i = trajectory_dict.get(str(well_i), pd.DataFrame())
                traj_k = trajectory_dict.get(str(well_k), pd.DataFrame())
                overlap_A, _, overlap_union = calculate_lateral_overlap_metrics(traj_i, traj_k)

                if overlap_type == "directional":
                    percent = overlap_A
                elif overlap_type == "union":
                    percent = overlap_union
                else:
                    raise ValueError("overlap_type must be 'directional' or 'union'")

                overlaps.append(percent)
                uwis.append(str(well_k))
                dirs.append(direction)
            else:
                overlaps.append(None)
                uwis.append(None)
                dirs.append(None)

            idxs.append(well_i)

        partial_df = pd.DataFrame({
            f"bound_{suffix}": overlaps,
            f"bound_{suffix}_uwi": uwis,
            f"bound_{suffix}_direction": dirs
        }, index=idxs)

        output = output.join(partial_df, how="left")

    # Reorder so that bound_1 is higher than bound_2
    reordered = output.copy()
    swap_mask = reordered["bound_2"].fillna(0) > reordered["bound_1"].fillna(0)

    for col in ["bound_1", "bound_1_uwi", "bound_1_direction"]:
        alt_col = col.replace("1", "2")
        reordered[col], reordered[alt_col] = (
            reordered[alt_col].where(swap_mask, reordered[col]),
            reordered[col].where(swap_mask, reordered[alt_col])
        )

    return reordered.reset_index()

In [40]:
lateral_adjacency_df_old = calculate_lateral_adjacency_bounds(
    spacing_df = df_spacing_new,
    trajectory_data = df_utm_lateral,  # pre-filtered lateral-only with 'uwi' and 'md'
    vertical_cutoff=125,
    horizontal_cutoff=1800
)

In [42]:
lateral_adjacency_df_new = calculate_lateral_adjacency_bounds(
    spacing_df = df_spacing_new,
    trajectory_data = df_utm_lateral,  # pre-filtered lateral-only with 'uwi' and 'md'
    vertical_cutoff=125,
    horizontal_cutoff=1800
)

In [43]:
lateral_adjacency_df_old

,well_i,bound_1,bound_1_uwi,bound_1_direction,bound_2,bound_2_uwi,bound_2_direction
0,42003073100100,NaN,None,None,NaN,None,None
1,42003442780000,NaN,None,None,NaN,None,None
2,42003452020000,NaN,None,None,NaN,None,None
3,42003455230000,NaN,None,None,NaN,None,None
4,42003455330000,NaN,None,None,NaN,None,None
...,...,...,...,...,...,...,...
23498,42501375310000,100.000000,42501375300000,W,99.305556,42501375320000,E
23499,42501375320000,99.305556,42501375310000,W,NaN,None,None
23500,42501375330000,NaN,None,None,NaN,None,None
23501,42501375540000,97.444946,42501368370000,E,96.812526,42501366520000,W


In [44]:
lateral_adjacency_df_new

,well_i,bound_1,bound_1_uwi,bound_1_direction,bound_2,bound_2_uwi,bound_2_direction
0,42003073100100,NaN,None,None,NaN,None,None
1,42003442780000,NaN,None,None,NaN,None,None
2,42003452020000,NaN,None,None,NaN,None,None
3,42003455230000,NaN,None,None,NaN,None,None
4,42003455330000,NaN,None,None,NaN,None,None
...,...,...,...,...,...,...,...
23498,42501375310000,100.000000,42501375300000,W,98.413305,42501375320000,E
23499,42501375320000,99.305556,42501375310000,W,NaN,None,None
23500,42501375330000,NaN,None,None,NaN,None,None
23501,42501375540000,88.160407,42501366520000,W,87.880331,42501368370000,E


In [53]:
df_spacing_new[(df_spacing_new['well_i']=='42003458530000')].sort_values(by='horizontal_dist', ascending=True)

,well_i,well_k,horizontal_dist,vertical_dist,3D_dist,drill_direction_i,drill_direction_k,direction_to_k_from_i
14100,42003458530000,42003459870100,78.268582,503.925,509.967035,NS,NS,S
14087,42003458530000,42003457690000,172.544372,79.890,190.141980,NS,NS,S
14101,42003458530000,42003460150000,229.965567,0.475,229.966058,NS,NS,S
14134,42003458530000,42003472710000,313.244848,2537.420,2556.681950,NS,NS,W
14117,42003458530000,42003467350000,334.331923,2585.955,2607.477919,NS,NS,N
...,...,...,...,...,...,...,...,...
14159,42003458530000,42003483780000,50909.577582,2675.380,50979.826870,NS,NS,E
14141,42003458530000,42003473980000,51457.545586,2534.720,51519.935979,NS,NS,E
14160,42003458530000,42003484760000,51856.552188,2681.916,51925.857511,NS,NS,E
14162,42003458530000,42003488190000,51933.042321,2698.095,52003.082614,NS,NS,E


In [56]:
df_MB_header[(df_MB_header['uwi'].str.startswith('4200345855')) | (df_MB_header['uwi']=='42003458530000')]

,uwi,lease_name,well_name,well_num,operator,rsv_cat,bench,first_prod_date,comp_date,hole_direction,surface_lat,surface_lon
3155,42003458550000,CARMICHAEL,CARMICHAEL 1504H,1504H,APA,01PDP,WICHITA ALBANY,2014-04-01,2014-04-22,H,32.209034,-102.799599
4574,42003458530000,CARMICHAEL,CARMICHAEL 1503H,1503H,APA,02PDNP,WICHITA ALBANY,2014-03-01,2014-03-12,H,32.203117,-102.799820


In [58]:
spacing_calculator._compute_normalized_midpoints(
    use_interpolation=True
)

C:\Users\Apoorva.Saxena\AppData\Local\Temp\ipykernel_26760\3509579785.py:99: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_prev = df.groupby("uwi").apply(lambda g: g.loc[g["row_index"] == g["prev_idx"].iloc[0]]).reset_index(drop=True)
C:\Users\Apoorva.Saxena\AppData\Local\Temp\ipykernel_26760\3509579785.py:100: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_next = df.groupby("uwi").apply(lambda g: g.lo

,x,y,tvd,latitude,longitude
uwi,,,,,
42003073100100,2.325885e+06,1.177279e+07,6115.849263,32.412634,-102.778288
42003442780000,2.368243e+06,1.177158e+07,4500.170526,32.406811,-102.641184
42003452020000,2.344760e+06,1.165908e+07,6392.059158,32.099113,-102.724962
42003455230000,2.317986e+06,1.165669e+07,7806.916947,32.094064,-102.811544
42003455330000,2.478920e+06,1.173691e+07,10132.131064,32.304367,-102.285659
...,...,...,...,...,...
42501375310000,2.262529e+06,1.201209e+07,5323.426500,33.073644,-102.968648
42501375320000,2.263184e+06,1.201214e+07,5323.472333,33.073729,-102.966506
42501375330000,2.294192e+06,1.201264e+07,5190.427979,33.073415,-102.865285
